# Uvod u inženjering upita
Inženjering upita je proces osmišljavanja i optimizacije upita za zadatke obrade prirodnog jezika. Uključuje odabir pravih upita, podešavanje njihovih parametara i procjenu njihove učinkovitosti. Inženjering upita je ključan za postizanje visoke točnosti i učinkovitosti u NLP modelima. U ovom odjeljku istražit ćemo osnove inženjeringa upita koristeći OpenAI modele za istraživanje.


### Vježba 1: Tokenizacija
Istražite tokenizaciju pomoću tiktokena, brzog open-source tokenizatora iz OpenAI-ja
Pogledajte [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) za više primjera.


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Vježba 2: Provjera postavki Github Models ključa

Pokreni donji kod kako bi provjerio je li tvoj Github Models endpoint ispravno postavljen. Kod samo isprobava jednostavan osnovni upit i provjerava dovršetak. Unos `oh say can you see` trebao bi se dovršiti otprilike s `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Vježba 3: Izmišljotine
Istražite što se događa kada zamolite LLM da vrati odgovore na upit o temi koja možda ne postoji, ili o temama o kojima možda nema saznanja jer su izvan njegovog unaprijed naučenog skupa podataka (novije informacije). Pogledajte kako se odgovor mijenja ako pokušate s drugačijim upitom ili drugim modelom.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Vježba 4: Temeljeno na uputama
Koristi varijablu "text" za postavljanje glavnog sadržaja 
i varijablu "prompt" za davanje upute povezane s tim glavnim sadržajem.

Ovdje tražimo od modela da sažme tekst za učenika drugog razreda osnovne škole


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Vježba 5: Složeni upit
Isprobajte zahtjev koji sadrži poruke sustava, korisnika i asistenta
Sustav postavlja kontekst asistenta
Poruke korisnika i asistenta pružaju kontekst za višestruku razmjenu

Obratite pažnju kako je osobnost asistenta postavljena na "sarkastičnu" u kontekstu sustava.
Pokušajte koristiti drugačiji kontekst osobnosti. Ili isprobajte drugačiji niz ulaznih/izlaznih poruka


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Vježba: Istražite svoju intuiciju
Gornji primjeri daju vam obrasce koje možete koristiti za stvaranje novih upita (jednostavnih, složenih, uputa itd.) – pokušajte osmisliti druge vježbe kako biste istražili neke od drugih ideja o kojima smo razgovarali, poput primjera, smjernica i još mnogo toga.



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge prevođenja [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni ljudski prijevod. Ne snosimo odgovornost za bilo kakva nesporazume ili pogrešna tumačenja koja proizlaze iz korištenja ovog prijevoda.
